In [1]:
import pandas as pd
import xarray as xr
from netCDF4 import Dataset as NetCDFFile

mySeries = pd.read_csv('/Users/bennysmith/Documents/Climate_Science/Research_with_Baylor/Datasets_from_Sherry/Oak_data.csv')
myDataset = xr.Dataset.from_dataframe(mySeries)

A = xr.Dataset()
A['datetime'] = myDataset['Date']+' '+myDataset['TIME']
# print(A['datetime'])
for i in myDataset.variables.keys():
    if i != 'Date' and i != 'TIME':
        A[i] = myDataset[i]
print(A)

MyDatasetnc = A.to_netcdf('/Users/bennysmith/Documents/Climate_Science/Research_with_Baylor/Datasets_from_Sherry/Oak_data_before.nc')

<xarray.Dataset>
Dimensions:   (index: 103)
Coordinates:
  * index     (index) int64 0 1 2 3 4 5 6 7 8 ... 94 95 96 97 98 99 100 101 102
Data variables:
    datetime  (index) object '5/26/15 7:51' '5/28/15 8:28' ... '8/29/18 10:25'
    ENT_1     (index) int64 31 20 1050 10 201 41 41 10 ... 10 10 84 10 10 10 121
    RAIN      (index) float64 0.0 0.0 0.29 0.0 0.0 0.0 ... 0.0 0.0 0.01 0.0 0.0
    P_DD      (index) int64 6 8 0 0 5 7 12 1 0 2 0 2 ... 3 8 11 3 5 0 2 0 0 5 7
    P_RD1     (index) float64 0.0 0.0 3.18 0.35 0.0 ... 0.0 0.99 0.02 0.0 0.0
    P_RD2     (index) float64 0.0 0.0 0.0 0.29 0.0 0.0 ... 1.02 0.0 0.0 0.0 0.0
    P_RD3     (index) float64 0.0 0.0 0.0 3.18 0.0 0.0 ... 0.63 0.0 0.99 0.0 0.0
    P_RD4     (index) float64 0.03 0.0 0.0 0.0 0.0 0.0 ... 0.72 0.0 0.0 0.25 0.0
    P_RD5     (index) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.01 0.0 0.0 0.01 0.0
    P_RD6     (index) float64 0.0 0.03 0.0 0.0 0.35 ... 0.02 1.02 0.0 0.02 0.25
    P_RD7     (index) float64 0.26 0.0 0.0 0.0 

In [8]:
import pandas as pd
import xarray as xr
import netCDF4 as nc
import numpy as np
mydirectory = '/Users/bennysmith/Documents/Climate_Science/Research_with_Baylor/Datasets_from_Sherry'
ncstart = nc.Dataset(mydirectory+'/Oak_data_before.nc')

mync = nc.Dataset('/Users/bennysmith/Documents/Climate_Science/Research_with_Baylor/Datasets_from_Sherry/Oak_data_test.nc', 'w', format = 'NETCDF4')
mync.title = 'RIDOH Statistical Model input data for Oakland Beach'
mync.history = 'Converted from Excel spreadsheet provided by the Rhode Island Department of Health'
mync.institution = 'Rhode Island Department of Health'
mync.references = 'Data from Sherry Poucher at the RIDOH Beach Monitoring Program, Sherry.Poucher@health.ri.gov'
date_dim = mync.createDimension('date', 103)

# Below I made a NetCDF file with the names of the abbreviated variables:
# mySeries = pd.read_csv('/Users/bennysmith/Documents/Climate_Science/Research_with_Baylor/Datasets_from_Sherry/Variable_definitions_Oakdata.csv')
# myDataset = xr.Dataset.from_dataframe(mySeries)
# variablesnc = myDataset.to_netcdf('/Users/bennysmith/Documents/Climate_Science/Research_with_Baylor/Datasets_from_Sherry/Variable_definitions_Oakdata.nc')

# Below I read in the variable names/abbreviations to use for the actual document I'm building
defs_abbrevs = nc.Dataset('/Users/bennysmith/Documents/Climate_Science/Research_with_Baylor/Datasets_from_Sherry/Variable_definitions_Oakdata.nc')
# print(defs_abbrevs.variables.keys())
Primary_Variable = defs_abbrevs.variables['Primary_Variable'][:]
Primary_Variable_Abbreviation = defs_abbrevs.variables['Primary_Variable_Abbreviation'][:]
Derived_Variable = defs_abbrevs.variables['Derived_Variable'][:]
Derived_Variable_Abbreviation = defs_abbrevs.variables['Derived_Variable_Abbreviation'][:]

variable_names = []
def variable_names_append(x):
    for i in x:
        variable_names.append(i)
variable_names_append(Primary_Variable)
variable_names_append(Derived_Variable)

variable_abbreviations = []
def variable_abbreviations_append(z):
    for i in z:
        variable_abbreviations.append(i)
variable_abbreviations_append(Primary_Variable_Abbreviation)
variable_abbreviations_append(Derived_Variable_Abbreviation)

print('hello123')
print(variable_names)
print('Hello world')
print(variable_abbreviations)
print('hello235')
print(ncstart.variables.keys())

print(len(variable_names))
print(len(variable_abbreviations))

overlap = []
for v in variable_abbreviations:
    if v in ncstart.variables.keys():
        overlap.append(v)
print(len(overlap))

print(len(ncstart.variables.keys()))
dtype_dict = {}
for i in ncstart.variables.keys():
    dtype_dict[i] = ncstart.variables[i].dtype
print(dtype_dict)

for variable in ncstart.variables.keys():
    if variable!='index':
        current_var = mync.createVariable(variable, dtype_dict[variable], ('date',))
        if variable in overlap:
            current_var.name_and_units = variable_names[variable_abbreviations.index(variable)]
mync.variables['ENT_1'].long_name = 'Enterococci Counts in colony-forming units'
mync.variables['ENT_1'].short_name = 'ECounts'
mync.variables['ENT_1'].standard_name = 'Enterococci_Counts_in_colony-forming_units'
mync.variables['ENT_1'].units = 'colony-forming unit'
for i in mync.variables.keys():
    mync.variables[i][:] = ncstart.variables[i][:]
mync.close()

# variables_Dataset = xr.Dataset.from_dataframe(mySeries)
# print(variables_Dataset.variables.keys())
# name = []
# abbreviation = []
# for i in variables_Dataset.variables['Primary Variable']:
#     name.append(i)
# for i in variables_Dataset.variables['Derived Variable']:
#     name.append(i)
# for j in variables_Dataset.variables['Primary Variable Abbreviation']:
#     abbreviation.append(j)
# for j in variables_Dataset.variables['Derived Variable Abbreviation']:
#     abbreviation.append(j)

# dtype_list = []
# for j in ncstart.variables.values():
#     dtype_list.append(j.dtype)
   
# keys_list = []
# for j in ncstart.variables.keys():
#     keys_list.append(j)

# for k in keys_list:
#     if k != 'index':
#         mync.createVariable(k, dtype_list[keys_list.index(k)], ('date',))

# mync.close()
# print('done')

hello123
['date', 'Rain_0', 'Notes', 'AM - High Time', 'AM-High Height (ft)', 'AM- Low Time', 'AM-Low Height(ft)', 'PM-High Time', 'PM-High Height (ft)', 'PM-Low Time', 'PM-Low Height', 'Water Temp (F)', 'Air Temp (F)', 'Wind Direction', 'Wind Speed (mph)', 'Weather-S', 'Surf-S', 'Tide-S', 'Vegetation in Water-S', 'Visible Sewage or Odor-S', 'Condition of Water-S', 'Seagull count-S', 'Clear Sky UV Index', 'Prev day clear sky UV', 'Cloudy Sky UV Index', 'Cloud Transmission', 'Solar Zenith Angle', 'Aerosol Transmission', 'Total Column Zone', 'Collection Time', '', 'Dependent Variables', 'ENT1 (station 1)', 'ENT2 (station 2)', 'ENT3 (station 3)', '', '', '', '', 'Number of Previous Dry Days', 'Rain_Day-1', 'Rain_Day-2', 'Rain_Day-3', 'Rain_Day-4', 'Rain_Day-5', 'Rain_Day-6', 'Rain_Day-7', 'Cumrain_0-1', 'Cumrain_0-2', 'Cumrain_0-3', 'Cumrain_0-4', 'Cumrain_0-5', 'Cumrain_0-6', 'Cumrain_0-7', 'Time btw high and low tide  ', 'Prevday time btw high and low tide', 'Time btw collection and  lo